<a href="https://colab.research.google.com/github/bachsykhang/StockPredictVinamilk/blob/main/Vinamilk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bước 1: Import thư viện

In [ ]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ
from sklearn.preprocessing import MinMaxScaler #chuẩn hóa dữ liệu
from keras.callbacks import ModelCheckpoint #lưu lại huấn luyện tốt nhất
from tensorflow.keras.models import load_model #tải mô hình

#các lớp để xây dựng mô hình
from keras.models import Sequential #đầu vào
from keras.layers import LSTM #học phụ thuộc
from keras.layers import Dropout #tránh học tủ
from keras.layers import Dense #đầu ra

#kiểm tra độ chính xác của mô hình
from sklearn.metrics import r2_score #đo mức độ phù hợp
from sklearn.metrics import mean_absolute_error #đo sai số tuyệt đối trung bình
from sklearn.metrics import mean_absolute_percentage_error #đo % sai số tuyệt đối trung bình

# Bước 2: Đọc dữ liệu

In [ ]:
#tạo quyền truy cập với google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# đọc dữ liệu từ file csv

df = pd.read_csv('/content/drive/MyDrive/Data Analysis (Project)/Data/VNM_PriceHistory.csv')

# Hiển thị lại DataFrame sau khi xóa
print(df)
# === Preprocess Data ===
# Chuyển đổi cột 'Ngày' sang định dạng datetime
df['Ngày'] = pd.to_datetime(df['Ngày'])

# Đặt 'Ngày' làm chỉ số (index)
df.set_index('Ngày', inplace=True)

# Kiểm tra dữ liệu bị thiếu
print("Missing values:")
print(df.isnull().sum())

# Loại bỏ các hàng có giá trị bị thiếu ở cột 'Đóng cửa'
df = df.dropna(subset=['Đóng cửa'])

# Lấy chuỗi thời gian từ cột 'Đóng cửa'
series = df['Đóng cửa']
# Xem thông tin cơ bản về dữ liệu
print("\nData Summary:")
print(series.describe())

## Khai phá dữ liệu

### Chỉ số Kỹ Thuật: Tính toán Moving Average (MA): làm mượt giá cổ phiếu để có thể thấy được xu hướng

In [ ]:
# Tính chỉ số Trung bình động đơn giản với kỳ hạn 50 và 200
df['MA_50'] = df['Đóng cửa'].rolling(window=50).mean()
df['MA_200'] = df['Đóng cửa'].rolling(window=200).mean()

# Vẽ biểu đồ

plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Đóng cửa'], linestyle='-', color='b', label='Đóng cửa')
plt.plot(df.index, df['MA_50'], linestyle='--', color='r', label='MA 50')
plt.plot(df.index, df['MA_200'], linestyle='--', color='g', label='MA 200')
plt.title('Biểu đồ Đường Giá Cổ Phiếu với MA 50 và MA 200')
plt.xlabel('Thời gian')
plt.ylabel('Giá Đóng cửa')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()

### Tính toán Volatility: Đo lường mức độ dao động của giá cổ phiếu để đánh giá rủi ro.Phát hiện các giai đoạn bất thường: biến động cao hoặc thấp.

In [ ]:
df['Volatility_30'] = df['Đóng cửa'].rolling(window=30).std()
df['Volatility_90'] = df['Đóng cửa'].rolling(window=90).std()

# Vẽ biểu đồ Volatility
plt.figure(figsize=(14, 7))
plt.plot(df['Volatility_30'], label='Volatility 30 ngày', color='orange', alpha=0.7)
plt.plot(df['Volatility_90'], label='Volatility 90 ngày', color='purple', alpha=0.7)
plt.title('Volatility của giá cổ phiếu')
plt.xlabel('Ngày')
plt.ylabel('Volatility')
plt.legend()
plt.grid()
plt.show()


# Bước 3: Mô tả dữ liệu

### Time series decomposition: Seasonal và trend

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Danh sách chu kỳ muốn thử nghiệm
periods = [52]  # Chu kỳ 3 tháng, 6 tháng, và 1 năm

for period in periods:
    decomposition = seasonal_decompose(series, model='additive', period=period)     # Decompose series với chu kỳ hiện tại

    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    plt.figure(figsize=(12, 12))
    plt.suptitle(f"Seasonal Decompose with Period = {period} weeks", fontsize=16)

    plt.subplot(411)
    plt.plot(series, label='Original', color='blue')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend, label='Trend', color='red')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal, label='Seasonality', color='green')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual, label='Residual', color='orange')
    plt.legend(loc='best')

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Chừa chỗ cho tiêu đề
    plt.show()


In [ ]:
# Plot theo năm
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Giả sử series là chuỗi dữ liệu (giá đóng cửa) có chỉ số thời gian
# Định nghĩa các năm cần phân tích
years_to_plot = ['2016', '2017', '2018']

# Chu kỳ (ví dụ: 52 tuần cho phân tích hàng năm)
period = 52

# Thực hiện Seasonal Decomposition
decomposition = seasonal_decompose(series, model='additive', period=period)

# Lấy các thành phần Trend, Seasonal, Residual
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Lọc dữ liệu theo năm cụ thể
for year in years_to_plot:
    trend_year = trend[trend.index.year == int(year)]
    seasonal_year = seasonal[seasonal.index.year == int(year)]
    residual_year = residual[residual.index.year == int(year)]
    original_year = series[series.index.year == int(year)]

    # Vẽ biểu đồ
    plt.figure(figsize=(12, 12))
    plt.suptitle(f"Seasonal Decomposition for Year {year}", fontsize=16)

    # Giá trị gốc
    plt.subplot(411)
    plt.plot(original_year, label='Original', color='blue')
    plt.title(f"Original Data - {year}")
    plt.legend(loc='best')

    # Xu hướng
    plt.subplot(412)
    plt.plot(trend_year, label='Trend', color='red')
    plt.title(f"Trend - {year}")
    plt.legend(loc='best')

    # Chu kỳ
    plt.subplot(413)
    plt.plot(seasonal_year, label='Seasonality', color='green')
    plt.title(f"Seasonality - {year}")
    plt.legend(loc='best')

    # Phần dư
    plt.subplot(414)
    plt.plot(residual_year, label='Residual', color='orange')
    plt.title(f"Residual - {year}")
    plt.legend(loc='best')

    # Hiển thị biểu đồ
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Định nghĩa các năm và các quý
years_to_plot = ['2016', '2017', '2018']
quarters = {'Q1': [1, 2, 3], 'Q2': [4, 5, 6], 'Q3': [7, 8, 9], 'Q4': [10, 11, 12]}

# Chu kỳ (ví dụ: 52 tuần)
period = 52

# Thực hiện Seasonal Decomposition
decomposition = seasonal_decompose(series, model='additive', period=period)

# Lấy các thành phần Trend, Seasonal, Residual
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Vẽ biểu đồ so sánh từng quý qua các năm
for quarter, months in quarters.items():
    plt.figure(figsize=(14, 10))
    plt.suptitle(f"Comparison of {quarter} for 2016, 2017, and 2018", fontsize=16)

    for year in years_to_plot:
        # Lọc dữ liệu theo năm và quý
        trend_quarter = trend[(trend.index.year == int(year)) & (trend.index.month.isin(months))]
        seasonal_quarter = seasonal[(seasonal.index.year == int(year)) & (seasonal.index.month.isin(months))]
        residual_quarter = residual[(residual.index.year == int(year)) & (residual.index.month.isin(months))]
        original_quarter = series[(series.index.year == int(year)) & (series.index.month.isin(months))]

        # Giá trị gốc
        plt.subplot(411)
        plt.plot(original_quarter, label=f'Original {year}', alpha=0.7)
        plt.title(f"Original Data - {quarter}")
        plt.legend(loc='best')

        # Xu hướng
        plt.subplot(412)
        plt.plot(trend_quarter, label=f'Trend {year}', alpha=0.7)
        plt.title(f"Trend - {quarter}")
        plt.legend(loc='best')

        # Chu kỳ
        plt.subplot(413)
        plt.plot(seasonal_quarter, label=f'Seasonality {year}', alpha=0.7)
        plt.title(f"Seasonality - {quarter}")
        plt.legend(loc='best')

        # Phần dư
        plt.subplot(414)
        plt.plot(residual_quarter, label=f'Residual {year}', alpha=0.7)
        plt.title(f"Residual - {quarter}")
        plt.legend(loc='best')

    # Hiển thị biểu đồ
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


## Xây dựng mô hình LSTM

In [ ]:
df

In [ ]:
# 3. Chuẩn hóa dữ liệu
scalers = {col: MinMaxScaler(feature_range=(0, 1)) for col in df.columns}
for col in df.columns:
    df[col] = scalers[col].fit_transform(df[[col]])

In [ ]:
df

In [ ]:
# Tạo chuỗi con cho LSTM
def create_sequences(data, time_steps=60):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])  # 60 ngày trước làm đầu vào
        y.append(data[i + time_steps, 0])  # Giá trị 'Đóng cửa' là nhãn
    return np.array(X), np.array(y)

scaled_data = df.values
time_steps = 60
X, y = create_sequences(scaled_data, time_steps)

In [ ]:
# Chia tập dữ liệu thành huấn luyện và kiểm tra
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Kiểm tra kích thước dữ liệu
print("Kích thước tập huấn luyện:", X_train.shape, y_train.shape)
print("Kích thước tập kiểm tra:", X_test.shape, y_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Xây dựng mô hình LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)  # Lớp cuối dự đoán giá 'Đóng cửa'
])

# Compile mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

# Hiển thị cấu trúc mô hình
model.summary()


In [ ]:
# Huấn luyện mô hình
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))


In [ ]:
# Lưu toàn bộ mô hình
model.save('/content/drive/MyDrive/Data Analysis (Project)/Data/lstm_stock_model.h5')
print("Mô hình đã được lưu dưới dạng lstm_stock_model.h5")


In [ ]:
from tensorflow.keras.models import load_model

# Đường dẫn đến tệp mô hình
model_path = '/content/drive/MyDrive/Data Analysis (Project)/Data/lstm_stock_model.h5'

# Tải lại mô hình với cấu hình tùy chỉnh
try:
    model = load_model(model_path)
    print("Mô hình đã được tải thành công!")
except Exception as e:
    print(f"Không thể tải mô hình. Lỗi: {e}")

In [ ]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ lỗi
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Dự báo trên tập kiểm tra
predictions = model.predict(X_test)

# Chuyển đổi giá trị về dạng ban đầu
predicted_prices = scalers['Đóng cửa'].inverse_transform(predictions)
real_prices = scalers['Đóng cửa'].inverse_transform(y_test.reshape(-1, 1))

# Lấy thời gian tương ứng với tập kiểm tra
test_dates = df.index[-len(X_test):]  # Chỉ số thời gian cho tập kiểm tra

# Vẽ biểu đồ so sánh
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(test_dates, real_prices, label='Giá thực tế', color='blue')
plt.plot(test_dates, predicted_prices, label='Giá dự báo', color='red')
plt.title('So sánh Giá Thực Tế và Giá Dự Báo (LSTM)')
plt.xlabel('Thời gian')
plt.ylabel('Giá cổ phiếu')
plt.legend()

# Hiển thị các mốc thời gian cụ thể (tháng/năm)
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


## Đánh giá mô hình: MSE/RMSE:
MSE/RMSE
Giá trị càng nhỏ, dự báo càng tốt.
RMSE dễ hiểu hơn vì cùng đơn vị với giá trị thực tế.

MAE:
Đơn vị tương tự giá trị thực tế, cho biết sai số trung bình.
MAPE:
Hiển thị sai số dưới dạng phần trăm.
Thích hợp để so sánh hiệu suất giữa các mô hình hoặc dữ liệu khác nhau.

R² Score:
Giá trị từ 0 đến 1.
Gần 1: Mô hình dự báo tốt.
Gần 0: Mô hình không giải thích được biến thiên.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Tính toán các thước đo
mse = mean_squared_error(real_prices, predicted_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(real_prices, predicted_prices)

# Hàm tính MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(real_prices, predicted_prices)
r2 = r2_score(real_prices, predicted_prices)

# In kết quả

print(f"R² Score: {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")




### Đánh giá
1. Mean Squared Error (MSE): 835,543.978
MSE đo lường sai số bình phương trung bình giữa giá trị dự báo và giá trị thực tế.
Giá trị MSE cao cho thấy sai số giữa các giá trị dự báo và thực tế lớn.
Với dữ liệu giá cổ phiếu, giá trị MSE khoảng 835,543 cho thấy mô hình dự đoán có sai số tương đối nhỏ nếu xét đơn vị đo giá (nghìn đồng).
2. Root Mean Squared Error (RMSE): 914.08
RMSE là căn bậc hai của MSE, có cùng đơn vị với giá trị thực tế (giá cổ phiếu trong trường hợp này).
Giá trị RMSE = 914.08 có nghĩa là, trung bình, dự báo của mô hình lệch khoảng 914 đồng so với giá thực tế.
Đánh giá:

RMSE thấp hơn so với giá cổ phiếu trung bình, chứng tỏ mô hình đang hoạt động tốt.
3. Mean Absolute Error (MAE): 686.98
MAE đo lường sai số tuyệt đối trung bình giữa giá trị dự báo và giá trị thực tế.
MAE = 686.98 có nghĩa là, trung bình, mô hình lệch khoảng 687 đồng so với giá thực tế.
Đánh giá:

MAE thường thấp hơn RMSE vì không bình phương sai số. Sai số tuyệt đối dưới 1,000 đồng là rất tốt nếu giá cổ phiếu trung bình cao (hàng chục nghìn đồng).
4. Mean Absolute Percentage Error (MAPE): 0.98%
MAPE đo lường sai số trung bình dưới dạng phần trăm.
MAPE = 0.98% nghĩa là, trung bình, sai số dự báo chỉ chiếm khoảng 0.98% giá trị thực tế.
Đánh giá:

Đây là một con số rất thấp, thể hiện mô hình dự đoán chính xác cao.
Đối với mô hình dự báo giá cổ phiếu, MAPE dưới 5% thường được coi là rất tốt.
5. R² Score: 0.9426
R² (Coefficient of Determination) đo lường tỷ lệ biến thiên trong giá trị thực tế được giải thích bởi mô hình.
R² = 0.9426 có nghĩa là mô hình giải thích được 94.26% biến động của giá cổ phiếu.
Đánh giá:

R² gần 1 là rất tốt, cho thấy mô hình dự đoán phù hợp với dữ liệu.

## Lưu lại giá trị dự báo

In [ ]:
from tensorflow.keras.models import load_model

# Đường dẫn đến tệp mô hình
model_path = '/content/drive/MyDrive/Data Analysis (Project)/Data/lstm_stock_model.h5'

# Tải lại mô hình với cấu hình tùy chỉnh
try:
    model = load_model(model_path)
    print("Mô hình đã được tải thành công!")
except Exception as e:
    print(f"Không thể tải mô hình. Lỗi: {e}")


In [ ]:
# Dự báo trên tập kiểm tra
predictions = model.predict(X_test)

# Chuyển đổi giá trị về dạng ban đầu
predicted_prices = scalers['Đóng cửa'].inverse_transform(predictions)
real_prices = scalers['Đóng cửa'].inverse_transform(y_test.reshape(-1, 1))

# Tạo DataFrame lưu giá trị thực tế và dự đoán
forecast_df = pd.DataFrame({
    'Ngày': df.index[-len(X_test):],  # Ngày tương ứng với tập kiểm tra
    'Giá Thực Tế': real_prices.flatten(),
    'Giá Dự Báo': predicted_prices.flatten()
})

# Hiển thị DataFrame kết quả
print(forecast_df.tail())

# Lưu dự báo vào DataFrame gốc
df['Giá Dự Báo'] = None
df.loc[df.index[-len(X_test):], 'Giá Dự Báo'] = predicted_prices.flatten()


In [ ]:
forecast_df

In [ ]:
# đọc dữ liệu từ file csv

df1 = pd.read_csv('/content/drive/MyDrive/Data Analysis (Project)/Data/VNM_PriceHistory.csv')

# Hiển thị lại DataFrame sau khi xóa
print(df1)
# === Preprocess Data ===
# Chuyển đổi cột 'Ngày' sang định dạng datetime
df1['Ngày'] = pd.to_datetime(df1['Ngày'])

# Đặt 'Ngày' làm chỉ số (index)
df1.set_index('Ngày', inplace=True)

# Kiểm tra dữ liệu bị thiếu
print("Missing values:")
print(df1.isnull().sum())

# Loại bỏ các hàng có giá trị bị thiếu ở cột 'Đóng cửa'
df1 = df1.dropna(subset=['Đóng cửa'])

# Lấy chuỗi thời gian từ cột 'Đóng cửa'
series = df1['Đóng cửa']
# Xem thông tin cơ bản về dữ liệu
print("\nData Summary:")
print(series.describe())

In [ ]:
# Tạo một cột "Giá Dự Báo" trong DataFrame và gán giá trị mặc định là None
df1['Giá Dự Báo'] = None

# Giả định rằng bạn đã có dữ liệu dự báo trong 'forecast_prices' và các ngày dự báo trong 'forecast_dates'
# Ví dụ minh họa:
import numpy as np
forecast_dates = forecast_df['Ngày'].values  # Ngày dự báo
forecast_prices = forecast_df['Giá Dự Báo'].values

# Gán giá trị dự báo vào cột "Giá Dự Báo" cho các ngày có dự báo
df1.loc[forecast_dates, 'Giá Dự Báo'] = forecast_prices

# Thay thế giá trị NaN trong "Giá Dự Báo" bằng giá trị từ cột "Đóng cửa"
df1['Giá Dự Báo'] = df1['Giá Dự Báo'].combine_first(df1['Đóng cửa'])
df1['Giá Dự Báo'] = pd.to_numeric(df1['Giá Dự Báo'], errors='coerce')
df1['Giá Dự Báo'] = df1['Giá Dự Báo'].round(1)

# Kiểm tra lại DataFrame để đảm bảo
print(df1[['Đóng cửa', 'Giá Dự Báo']].head())  # Kiểm tra một vài giá trị đầu
print(df1[['Đóng cửa', 'Giá Dự Báo']].tail())  # Kiểm tra một vài giá trị cuối

# Xác nhận số lượng dòng trong cả hai cột là như nhau
print(f"Số dòng của 'Đóng cửa': {len(df1['Đóng cửa'])}")
print(f"Số dòng của 'Giá Dự Báo': {len(df1['Giá Dự Báo'])}")


In [ ]:
# In toàn bộ giá trị của cột "Đóng cửa" và "Giá Dự Báo"
pd.set_option('display.max_rows', None)  # Hiển thị tất cả các dòng
print(df1[['Đóng cửa', 'Giá Dự Báo']])  # In toàn bộ cột "Đóng cửa" và "Giá Dự Báo"



In [ ]:
# Xóa cột MA_50_Dự Báo và MA_200_Dự Báo khỏi DataFrame df1
df1.drop(columns=['MA_50_Đóng Cửa', 'MA_200_Đóng Cửa'], inplace=True)

# Kiểm tra lại DataFrame để đảm bảo các cột đã bị xóa
df1.columns
# Lưu DataFrame df1 thành tệp CSV
#df1.to_csv(('/content/drive/MyDrive/Data Analysis (Project)/Data/Prediction_Price_LSTM.csv'), index=True)  # Lưu file CSV với index



### Tính các chỉ số kỹ thuật

In [ ]:
# Tính MA 50 và MA 200 trên Giá Dự Báo
df1['MA_50_Dự Báo'] = df1['Giá Dự Báo'].rolling(window=50).mean()
df1['MA_200_Dự Báo'] = df1['Giá Dự Báo'].rolling(window=200).mean()

# Tính MA 50 và MA 200 trên Giá Đóng Cửa
#df1['MA_50_Đóng Cửa'] = df1['Đóng cửa'].rolling(window=50).mean()
#df1['MA_200_Đóng Cửa'] = df1['Đóng cửa'].rolling(window=200).mean()

# In MA 50 và MA 200 của Giá Dự Báo
print("MA 50 và MA 200 của Giá Dự Báo:")
print(df1[['MA_50_Dự Báo', 'MA_200_Dự Báo']].dropna().head())

# In MA 50 và MA 200 của Giá Đóng Cửa
#print("\nMA 50 và MA 200 của Giá Đóng Cửa:")
#print(df1[['MA_50_Đóng Cửa', 'MA_200_Đóng Cửa']].dropna().head())

# Vẽ biểu đồ MA 50 và MA 200 của Giá Dự Báo
plt.figure(figsize=(12, 6))

# Vẽ giá dự báo
plt.plot(df1.index, df1['Giá Dự Báo'], label='Giá Dự Báo', color='blue', alpha=0.7)

# Vẽ đường MA 50 và MA 200
plt.plot(df1.index, df1['MA_50_Dự Báo'], label='MA 50 Dự Báo', color='green', linestyle='--')
plt.plot(df1.index, df1['MA_200_Dự Báo'], label='MA 200 Dự Báo', color='red', linestyle='--')

# Tùy chỉnh biểu đồ
plt.title('MA 50 và MA 200 của Giá Dự Báo')
plt.xlabel('Thời gian')
plt.ylabel('Giá trị')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Tính độ lệch chuẩn (Volatility) dựa trên Giá Dự Báo
df1['Volatility_30_Dự Báo'] = df1['Giá Dự Báo'].rolling(window=30).std()
df1['Volatility_90_Dự Báo'] = df1['Giá Dự Báo'].rolling(window=90).std()


plt.figure(figsize=(12, 6))

# Vẽ giá dự báo
#plt.plot(df1.index, df1['Giá Dự Báo'], label='Giá Dự Báo', color='blue', alpha=0.5)

# Vẽ độ lệch chuẩn 30 ngày và 90 ngày
plt.plot(df1.index, df1['Volatility_30_Dự Báo'], label='Volatility 30 Ngày (Dự Báo)', color='orange')
plt.plot(df1.index, df1['Volatility_90_Dự Báo'], label='Volatility 90 Ngày (Dự Báo)', color='red')

# Tùy chỉnh biểu đồ
plt.title('Volatility (30 Ngày và 90 Ngày) của Giá Dự Báo')
plt.xlabel('Thời gian')
plt.ylabel('Volatility')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Loại bỏ giá trị NaN trước khi phân tích
forecast_data = df1['Giá Dự Báo'].dropna()

# Phân tích Seasonal và Trend từ Giá Dự Báo
decomposition = seasonal_decompose(forecast_data, model='additive', period=52)  # Chu kỳ 52 tuần

# Lưu lại các thành phần Trend và Seasonal vào DataFrame df1
df1['Trend_Dự Báo'] = decomposition.trend
df1['Seasonal_Dự Báo'] = decomposition.seasonal

# Vẽ biểu đồ các thành phần Trend và Seasonal
plt.figure(figsize=(12, 8))

# Biểu đồ Trend
plt.subplot(2, 1, 1)
plt.plot(df1.index, df1['Trend_Dự Báo'], label='Trend (Dự Báo)', color='orange')
plt.title('Trend của Giá Dự Báo')
plt.xlabel('Thời gian')
plt.ylabel('Trend')
plt.legend()
plt.grid()

# Biểu đồ Seasonal
plt.subplot(2, 1, 2)
plt.plot(df1.index, df1['Seasonal_Dự Báo'], label='Seasonality (Dự Báo)', color='green')
plt.title('Seasonality của Giá Dự Báo')
plt.xlabel('Thời gian')
plt.ylabel('Seasonal')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


## Đánh giá trọng số của các feature trong việc đưa ra các quyết định mua bán giữ

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import shap
import matplotlib.pyplot as plt

# Load the dataset
file_path = "/content/drive/MyDrive/Data Analysis (Project)/Data/Prediction_Price_LSTM.csv"
df = pd.read_csv(file_path, parse_dates=["Ngày"], dayfirst=True)

# Rename columns for easier processing
df.columns = ["Date", "Close", "Open", "High", "Low", "Volume", "Percent_Change", "Forecast_Price"]

# Ensure 'Date' column is in datetime format
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Check for invalid dates
if df["Date"].isna().sum() > 0:
    print("Some rows in the 'Date' column could not be parsed. Check the data format.")
    print(df[df["Date"].isna()])
    df = df.dropna(subset=["Date"])  # Drop invalid rows if necessary

# Sort by date
df = df.sort_values(by="Date")

# Calculate Moving Average (MA)
df["MA50"] = df["Close"].rolling(window=50, min_periods=1).mean()
df["MA200"] = df["Close"].rolling(window=200, min_periods=1).mean()
df["MA"] = (df["MA50"] + df["MA200"]) / 2

# Calculate Volatility (Standard Deviation of Closing Price)
df["Volatility"] = df["Close"].rolling(window=14, min_periods=1).std()

# Calculate Trend (Difference between Forecast and Close price)
df["Trend"] = df["Forecast_Price"] - df["Close"]

# Add a simplified seasonal index (e.g., monthly average)
df["Seasonality"] = df["Close"].groupby(df["Date"].dt.month).transform("mean")

# Create decision column (0 = Hold, 1 = Buy, -1 = Sell)
df["Decision"] = np.where(df["Forecast_Price"] > df["Close"] * 1.01, 1,
                          np.where(df["Forecast_Price"] < df["Close"] * 0.99, -1, 0))

# Prepare data for training
features = ["MA", "Volatility", "Trend", "Seasonality"]
X = df[features]
y = df["Decision"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Use SHAP to explain feature importance
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Ensure X_test is a DataFrame with proper feature names
if not isinstance(X_test, pd.DataFrame):
    X_test = pd.DataFrame(X_test, columns=features)

# Plot SHAP summary
shap.summary_plot(shap_values, features=X_test, feature_names=X_test.columns)

# Save the processed data for further inspection
processed_file_path = "/content/drive/MyDrive/Data Analysis (Project)/Data/Prediction_Price_LSTM(2).csv"
df.to_csv(processed_file_path, index=False)

print("Processed data saved at:", processed_file_path)


In [ ]:
# Group SHAP values by decisions: Buy (1), Hold (0), Sell (-1)
decision_categories = [-1, 0, 1]  # Sell, Hold, Buy
shap_values_by_decision = {cat: [] for cat in decision_categories}

# Split SHAP values and corresponding rows based on decision
for i, decision in enumerate(df["Decision"].iloc[X_test.index]):
    shap_array = shap_values[i].values  # Extract SHAP values as array
    if len(shap_array) == len(X_test.columns):  # Ensure consistent length
        shap_values_by_decision[decision].append(shap_array)
    else:
        print(f"Skipping SHAP value at index {i} due to length mismatch.")

# Aggregate feature importances for each decision
feature_importance_by_decision = {}
for decision, values in shap_values_by_decision.items():
    if values:
        # Convert list of SHAP values to a matrix for calculation
        decision_shap_values = np.vstack(values)  # Stack arrays into a matrix
        avg_importance = np.mean(np.abs(decision_shap_values), axis=0)  # Mean absolute SHAP values
        feature_importance_by_decision[decision] = pd.DataFrame({
            "Feature": X_test.columns[:len(avg_importance)],  # Ensure matching length
            "Importance": avg_importance
        }).sort_values(by="Importance", ascending=False)

# Plot and print results for each decision
for decision, importance_df in feature_importance_by_decision.items():
    decision_label = "Sell" if decision == -1 else "Hold" if decision == 0 else "Buy"
    # Plot SHAP summary
    plt.figure()
    plt.bar(importance_df["Feature"], importance_df["Importance"])
    plt.title(f"Feature Importance for {decision_label} Decision")
    plt.ylabel("Average SHAP Value")
    plt.xlabel("Feature")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print feature importances
    print(f"\nFeature Importance for {decision_label} Decision:")
    print(importance_df)


In [ ]:
# Calculate min and max SHAP values across all decisions
shap_min = np.min([np.min(np.abs(values)) for cat, values in shap_values_by_decision.items() if values])
shap_max = np.max([np.max(np.abs(values)) for cat, values in shap_values_by_decision.items() if values])

# Normalize SHAP values and update feature importance
normalized_feature_importance_by_decision = {}
for decision, values in shap_values_by_decision.items():
    if values:
        # Stack SHAP values into a matrix
        decision_shap_values = np.vstack(values)  # Stack arrays into a matrix
        avg_importance = np.mean(np.abs(decision_shap_values), axis=0)  # Mean absolute SHAP values

        # Normalize SHAP values
        normalized_avg_importance = (avg_importance - shap_min) / (shap_max - shap_min)

        # Store normalized importance
        normalized_feature_importance_by_decision[decision] = pd.DataFrame({
            "Feature": X_test.columns[:len(normalized_avg_importance)],  # Ensure matching length
            "Normalized Importance": normalized_avg_importance
        }).sort_values(by="Normalized Importance", ascending=False)

# Print normalized feature importance for each decision
for decision, importance_df in normalized_feature_importance_by_decision.items():
    decision_label = "Sell" if decision == -1 else "Hold" if decision == 0 else "Buy"
    print(f"\nNormalized Feature Importance for {decision_label} Decision:")
    print(importance_df)
